In [13]:
# Check these before running
# KPUM Workstation
sourcedatabasefolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata'
studydir = ''
# Finn's Laptop
#sourcedatabasefolder = '/mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata'
sourcedatabasefolder = '/mnt/s/Research/Projects/KPUM_NODDI/Data/sourcedata'
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'

# nifti base folder to go within studydir
outputniftibasefolder = 'nifti'

# tsv-files to keep track of in studydir/outputniftibasefolder
subjecttrackerfile = 'Subject_Tracker_for_sourcedata2nifti.tsv'

###################################################################################
# User input

# Participant details
subject = '0XX'
session = 'MR1'

# Define good DCM datafolder as input (and if no data is not good enough or does not exist, leave blank)

## ANATOMY
# High-resolution T1w MPRAGE
t1w_mprage = ''
# FLAIR
flair = ''
# T2star
t2star = ''
# T2w Cor
t2wtra = ''
# T2w Cor
t2wcor = ''
# SWI (if available)
swi = ''
# SWI minIP (if available)
swiminip = ''
# High-resolution T2w MCRIB (if available)
t2w_mcrib = 's05_T2W_MCRIB'

## DIFFUSION
# DKI dir-AP data
dki_dirAP = 's17_DKI_2.5mm_NODDI_dir-AP_79dir_0_1000_2000_DFC_MIX'
# DKI dir-PA data (if available)
dki_dirPA = 's16_DKI_2.5mm_NODDI_dir-PA_2dir_0_DFC_MIX'


###################################################################################



In [ ]:

# Define local functions
def dicom2nifti(inputdicomdir, outputniftidir, outfilebase):
    # Convert DCMs to NIfTIs using dcm2niix
    import subprocess
    from datetime import datetime
    
    print("Start converting DCMs")
    starttime=datetime.now()
    p=subprocess.Popen(f"dcm2niix -a y -d 1 -b y -w 1 -o {outputniftidir} -f {outfilebase} {inputdicomdir}", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()  
    #This makes the wait possible
    p_status = p.wait()
    #This will give you the output of the command being executed
    #print(f"Command output: {output}")
    endtime=datetime.now()
    print("Done converting DCMs. Elapsed time:", endtime-starttime)


# Create conversion file sourcedata2nifti.tsv
import os, subprocess
import pandas as pd

# Define I/O folders
sourcedatainputfolder = os.path.join(sourcedatabasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
niftioutputfolder = os.path.join(studydir, outputniftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
if not os.path.exists(niftioutputfolder): # then make this directory
    os.makedirs(niftioutputfolder)

# Create list for matching file transfers
sourcedatalist = [t1w_mprage, 
                  t2w_mcrib,
                  t2wtra,
                  t2wcor,
                  flair,
                  t2star,
                  swi,
                  swiminip,
                  dki_dirAP,
                  dki_dirPA ] 
niftidatalist = [os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mprage_T1w.nii'), 
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mcrib_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-tra_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-cor_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_FLAIR.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-t2star_T2starw.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-swi_T2starw.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-swi_rec-mnip_T2starw.nii'),
                 os.path.join('dwi',f'sub-{subject}_ses-{session}_dir-AP_dwi.nii'),
                 os.path.join('fmap',f'sub-{subject}_ses-{session}_dir-PA_epi.nii'),
                 ]
# and make this into a DataFrame
sample = {'input': sourcedatalist, 
          'output': niftidatalist,
          'converted': '',
          'comments': ''}
df = pd.DataFrame(sample)

# Locate only the files that are going to be transferred and set this entry to Pending (i.e. not have entry '' in sourcedatalist)
df.loc[df['input'] != '', ['converted']] = 'Pending'
# and save to tsv-file in niftioutputfolder
df.loc[df['converted'] == 'Pending'].to_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t", index=False)

# Now loop over this Pending entries in df
# Read df again from newly written file
df=pd.read_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t")

# and perform conversion using dcm2niix
for ind, row in df.loc[df['converted'] == 'Pending'].iterrows() :
    rowdata = row['input']
    inputdicomdir = os.path.join(sourcedatainputfolder, df['input'][ind])
    outfilebase = os.path.splitext(df['output'][ind])[0] # workaround to get rid of file-extension
    print('Start converting DCMs in', inputdicomdir ,'and save in', os.path.join(niftioutputfolder, outfilebase))
    # Convert DCMs using local function 
    dicom2nifti(inputdicomdir, niftioutputfolder, outfilebase)
     
    # Update tracker file
    df.loc['converted'][ind] = 'Done'
    df=pd.read_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t")
        
    print('Finished converting DCMs in', inputdicomdir,'and save in', os.path.join(niftioutputfolder, outfilebase))
    print()
        
